# HTLC – A Classical Smart Contract
#### 06.2 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write a Hash Time Locked Contract
* Use secret (hashed) passwords
* Use time delays
* Limits of passwords on the blockchain

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [6]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [7]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [8]:
from pyteal import *

In [9]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [10]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Quick check of asset holdings, otherwise go to ...
- https://bank.testnet.algorand.network
- https://testnet.algoexplorer.io/dispenser

In [11]:
ascset_holdings_df(algod_client,Alice['public'])

,amount,unit,asset-id,name,decimals
0,30.843996,ALGO,0,Algorand,6
1,200.000000,USDC,10458941,USDC,6
2,0.100000,WSC,66504861,WSC coin,2
3,0.000000,WSC,66505040,WSC coin,2
4,100.000000,ALICE,66712019,Alice's Tempcoin,1
5,100.000000,ALICE,66712340,Alice's Tempcoin,1
6,1.000000,ALICEART,69394953,Alice's First Portrait 001,0
7,0.000000,VtC,70161280,VoteCoin,2
8,0.000000,VtC,70166124,VoteCoin,2


#### Check Purestake API

In [12]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19812241


## The Cash Machine revisited

#### Remember
* Alice created a *Cash Machine* contract that requires a password
* Bob has to know the password to withdraw money
* Alice must give the following to Bob
  * The **address** of the smart contract
  * The **code of the smart signature** $\leftarrow$ this is actually a **problem**

In [91]:
# Looking again at the Cash Machine

# Step 1: Conditions as a PyTeal
cashmachine_pyteal = ( 
    Txn.note() == Bytes('{"Very long and very secret message"}') 
    )

# Step 2-3: Compile PyTeal -> Teal -> Bytecode for AVM
cashmachine_teal = compileTeal(cashmachine_pyteal, Mode.Signature, version=3)

# compile Teal -> Bytecode
Cashmachine = algod_client.compile(cashmachine_teal)
print("Alice sends these two items to Bob")
print("Compiled smart signature:", Cashmachine['result'])
print("Address of smart signature: ", Cashmachine['hash'])

Alice sends these two items to Bob
Compiled smart signature: AyYBJXsiVmVyeSBsb25nIGFuZCB2ZXJ5IHNlY3JldCBtZXNzYWdlIn0xBSgS
Address of smart signature:  6356RZLJANV4OT7WX2N25NVKR2U22XV3G3ZVK6J44SOZWZAUC75A3GXRSE


#### So what exactly ...

is inside this smart signature code?

In [92]:
# The code is bascially a base-64 encoding of the TEAL program
base64.b64decode(Cashmachine['result'])

b'\x03&\x01%{"Very long and very secret message"}1\x05(\x12'

## The Hash lock contract

#### Better
* Store the **hash** of the password

#### Step 1: Create a hash of the password

In [13]:
secret_password = 'WSC secret'
hash = hashlib.sha256( secret_password.encode() )
print(hash.hexdigest())                                 # The hash is HEX encoded

76241abc874467ec6cb2b9cdf60c712bfd213425fb391a5bef81b5efd22138d5


#### Step 2: Use the hashed password in PyTeal condition

In [16]:
hashlock_cond = ( 
    Sha256(Txn.note()) == Bytes(hash.hexdigest()) 
    )

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )

hashlock_pyteal = And(
    hashlock_cond, 
    random_cond, 
    fee_cond, 
    safety_cond
    )

In [15]:
# Alternatively if you have the hashed value in HEX = base16 representation
# Convert to Bytes using the following command
# Note: the "0x" at the beginning of the hash is optional
hashlock_cond = (
    Sha256(Txn.note()) == Bytes("base16","76241abc874467ec6cb2b9cdf60c712bfd213425fb391a5bef81b5efd22138d5")
     )

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )

hashlock_pyteal = And (hashlock_cond, random_cond, fee_cond, safety_cond)

#### Step 3-4: Compile

In [100]:
# Compile PyTeal -> Teal -> Bytecode for AVM
hashlock_teal = compileTeal(hashlock_pyteal, Mode.Signature, version=3)
print(hashlock_teal)

# Step 3: Compile Teal 
Hashlock = algod_client.compile(hashlock_teal)
print("Alice sends these two items to Bob")
print("Compiled smart signature:", Hashlock['result'])
print("Address of smart signature: ", Hashlock['hash'])

#pragma version 3
txn Note
sha256
byte 0x76241abc874467ec6cb2b9cdf60c712bfd213425fb391a5bef81b5efd22138d5
==
int 2892832491
int 2892832491
==
&&
Alice sends these two items to Bob
Compiled smart signature: AyAB67204womASB2JBq8h0Rn7Gyyuc32DHEr/SE0Jfs5GlvvgbXv0iE41TEFASgSIiISEA==
Address of smart signature:  TH2JUTSC7DPJGMKBRWKXKICX7T5WUMTUVJRS2CGAV3JKROZ5MZQD74AQBI


#### Step 5: Quick check: can Bob crack the code?

In [101]:
print(base64.b64decode(Hashlock['result']))
# BTW: can you find the hash of the password?
print(base64.b64decode(Hashlock['result']).hex())

b'\x03 \x01\xeb\xbd\xb4\xe3\n&\x01 v$\x1a\xbc\x87Dg\xecl\xb2\xb9\xcd\xf6\x0cq+\xfd!4%\xfb9\x1a[\xef\x81\xb5\xef\xd2!8\xd51\x05\x01(\x12""\x12\x10'
032001ebbdb4e30a26012076241abc874467ec6cb2b9cdf60c712bfd213425fb391a5bef81b5efd22138d5310501281222221210


#### Step 6: Alice is funding the Smart Signature

In [102]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Hashlock['hash'], amt=amt)

# Step 2+3: sign and send
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

print('http://testnet.algoexplorer.io/tx/'+txid)

Current round is  19808154.
Waiting for round 19808154 to finish.
Waiting for round 19808155 to finish.
Transaction QKILDI76OSHFHBLQZPRVUED2THV6JRFDVQDD56PSLJRYIIEQPPEA confirmed in round 19808156.
http://testnet.algoexplorer.io/tx/QKILDI76OSHFHBLQZPRVUED2THV6JRFDVQDD56PSLJRYIIEQPPEA


#### Step 7: Bob asks the smart signature to sign a transaction with the correct password

In [105]:
# correct password
withdrawal_amt = 100000
my_note        = 'WSC secret'

# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- OK!!
txn = PaymentTxn(sender=Hashlock['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                note = my_note.encode(),
                close_remainder_to = Bob['public'])

# Step 2: sign TX <---- This step is different!
encodedProg = Hashlock['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19808294.
Waiting for round 19808294 to finish.
Waiting for round 19808295 to finish.
Transaction SESVMSGKXZY6JIFJH47743IABRGERJO4EJCTSE7H62ZYG5XJEM5Q confirmed in round 19808296.


#### Step 8: Check the "Note" field on Algoexplorer
The password is now out in the open ... we cannot do anything about it

In [106]:
print('http://testnet.algoexplorer.io/tx/'+txid)

http://testnet.algoexplorer.io/tx/SESVMSGKXZY6JIFJH47743IABRGERJO4EJCTSE7H62ZYG5XJEM5Q


## Things that do not work
* Using the wrong password
* Sending a hash of the pasword instead of the password (because it will be hashed again)

## The Timelock contract

* Money can only be accessed after a certain amount of time
* *Examples:* vesting periods, deposits

##### Step 1: Fomulate PyTeal conditions

In [17]:
# prepare time condition
start_round = algod_client.status()["last-round"] 
time_cond = (
    Txn.first_valid() > Int(start_round+10)           # Earliest payout after 10 rounds from "now"
)

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )
    
timelock_pyteal = And(
    time_cond, 
    random_cond, 
    fee_cond, 
    safety_cond
    )

##### Step 2-3: Compile

In [18]:
# Step 2: Compile PyTeal -> Teal
timelock_teal = compileTeal(timelock_pyteal, Mode.Signature, version=3)
print(timelock_teal)

# Step 3: Teal -> Bytecode for AVM
Timelock = algod_client.compile(timelock_teal)
print("Smart signature addr: ", Timelock['hash'])
print("Smart signature code: ", Timelock['result'])

#pragma version 3
txn FirstValid
int 19812266
>
int 895003614
int 895003614
==
&&
txn Fee
int 1000
<=
&&
global GroupSize
int 1
==
txn TypeEnum
int pay
==
&&
txn RekeyTo
global ZeroAddress
==
&&
txn CloseRemainderTo
global ZeroAddress
==
&&
&&
Smart signature addr:  34UPSBBLA4Q6PP7XQIRC7GKXC4T2FXSALQ4VXGC4IUEDVQY3MAES7SS63M
Smart signature code:  AyAEqp+5Cd7X4qoD6AcBMQIiDSMjEhAxASQOEDIEJRIxECUSEDEgMgMSEDEJMgMSEBA=


##### Step 4: Alice funds and deploys the Smart Signature

In [117]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.5*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Timelock['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19808459.
Waiting for round 19808459 to finish.
Waiting for round 19808460 to finish.
Transaction KAKY5KFKTW3EEF6NAVJDRRIQIJJKNXL6VCSF7ENFXPBMRVJTM2BQ confirmed in round 19808461.


##### Step 5: Bob asks the smart signature to authorize a transaction
* Be quick! 
* The transaction will fail first
* But it will work after 10 rounds (approx 20-40 seconds)

In [120]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)

txn = PaymentTxn(sender=Timelock['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Timelock['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19808472.
Waiting for round 19808472 to finish.
Waiting for round 19808473 to finish.
Waiting for round 19808474 to finish.
Transaction FSUBHUUWGAIPN3IUWTHUMOHBJ4IYOL33QYB4QL2SZZKR63MDO6SA confirmed in round 19808474.


## The HTLC contract
* A classical Smart Contract
* Putting Hash Contract and Timelock contract together

##### Step 1: All conditions in PyTeal

In [19]:
# Prepare hash condition
import hashlib
secret_password = 'WSC secret'
pwdhash = hashlib.sha256( secret_password.encode() )
hash_cond =  (
    Sha256(Txn.note()) == Bytes(pwdhash.hexdigest())
)

# prepare time conditoin
start_round = algod_client.status()["last-round"] 
time_cond = (
    Txn.first_valid() > Int(start_round+60)                    # Earliest payout after 60 rounds from "now"
)

# prepare reandom condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = Txn.fee() <= Int(1000)

safety_cond = And(
    Txn.type_enum() == TxnType.Payment,
    Txn.close_remainder_to() == Global.zero_address(),
    Txn.rekey_to() == Global.zero_address(),
    )
    
htlc_pyteal = And(
    Or(hash_cond, time_cond), 
    random_cond, 
    fee_cond, 
    safety_cond
    )

##### Step 2-3: Compile

In [20]:
# Step 2: Compile PyTeal -> Teal
htlc_teal = compileTeal(htlc_pyteal, Mode.Signature, version=3)
print(htlc_teal)

# Step 3: Teal -> Bytecode for AVM
Htlc = algod_client.compile(timelock_teal)
print("Smart signature addr: ", Htlc['hash'])
print("Smart signature code: ", Htlc['result'])

#pragma version 3
txn Note
sha256
byte "76241abc874467ec6cb2b9cdf60c712bfd213425fb391a5bef81b5efd22138d5"
==
txn FirstValid
int 19812321
>
||
int 625620676
int 625620676
==
&&
txn Fee
int 1000
<=
&&
txn TypeEnum
int pay
==
txn CloseRemainderTo
global ZeroAddress
==
&&
txn RekeyTo
global ZeroAddress
==
&&
&&
Smart signature addr:  34UPSBBLA4Q6PP7XQIRC7GKXC4T2FXSALQ4VXGC4IUEDVQY3MAES7SS63M
Smart signature code:  AyAEqp+5Cd7X4qoD6AcBMQIiDSMjEhAxASQOEDIEJRIxECUSEDEgMgMSEDEJMgMSEBA=


##### Step 4: Alice funds and deploys the Smart Signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.5*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Htlc['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 5a: Bob asks the smart signature to authorize a transaction with password

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)
my_note        = 'WSC secret'
note           = my_note.encode()

txn = PaymentTxn(sender=Htlc['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                 note=note)

# Step 2: sign TX <---- This step is different!
encodedProg = Htlc['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)


##### –OR– Step 5b: Bob waits for the timelock to expire

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)
txn = PaymentTxn(sender=Htlc['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Htlc['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

* Check the transactions of the Smart Signature
* Once the password has been used, it is out in the public

In [ ]:
print('https://testnet.algoexplorer.io/address/'+htlc['hash'])